# Neural Architecture Search With Ax

We will follow the tutorial [HPO for PyTorch](https://ax.dev/tutorials/tune_cnn.html). We will learn to use Ax to configure and search a search space.

We'll work on the problem described in the tutorial. It tunes the widths of two hidden layers, the learning rate, the dropout probability, the batch size, and the number of training epochs. The search objective is to find a good trade off between model performance and model size.

Requirements for execution on cloud:
- [] Update TOTAL_TRIALS at [Choose a Generation Strategy](#7-choose-a-generation-strategy)

# 1. Import Modules

## 1.1. Configure Toolbox

In [3]:
! git clone https://github.com/NareshPS/doi-ml-toolbox.git
! mv doi-ml-toolbox/torch/toolbox .
! rm -rf doi-ml-toolbox

source: Error encountered while sourcing file '/Users/broxoli/venv-tensorflow/bin/activate.fish':
source: No such file or directory
Cloning into 'doi-ml-toolbox'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 42 (delta 12), reused 39 (delta 11), pack-reused 0
Receiving objects: 100% (42/42), 892.46 KiB | 2.26 MiB/s, done.
Resolving deltas: 100% (12/12), done.
source: Error encountered while sourcing file '/Users/broxoli/venv-tensorflow/bin/activate.fish':
source: No such file or directory
mv: rename doi-ml-toolbox/torch/toolbox to ./toolbox: Directory not empty
source: Error encountered while sourcing file '/Users/broxoli/venv-tensorflow/bin/activate.fish':
source: No such file or directory


## 1.2. Import Modules

In [4]:
import torch
import numpy as np

from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render, init_notebook_plotting
from ax.utils.tutorials.cnn_utils import load_mnist, train, evaluate, CNN
from toolbox import utils

# 2. Device Selection

In [5]:
device = utils.get_device()

print(f'Device: {device}')

Device: mps


# 3. Data Preparation

## 3.1. Hyperparameters

In [6]:
BATCH_SIZE = 512

## 3.2. Create Dataloaders for MNIST

In [7]:
train_loader, valid_loader, test_loader = load_mnist(batch_size=BATCH_SIZE)

100%|██████████| 9912422/9912422 [00:01<00:00, 7530582.11it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 36464688.09it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 2324087.38it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 9028686.62it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



# 4. Define Optimization Function

In [14]:
def train_and_evaluate(parameters):
    net = CNN()
    net = train(net=net, train_loader=train_loader, parameters=parameters, dtype=torch.float, device=device)
    return evaluate(
        net=net,
        data_loader=valid_loader,
        dtype=torch.float,
        device=device,
    )

In [15]:
train_and_evaluate(dict(lr=0.01, momentum=0.9))

0.104

# 5. Optimization Loop

In [17]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "lr", "type": "range", "bounds": [1e-6, 0.4], "log_scale": True},
        {"name": "momentum", "type": "range", "bounds": [0.0, 1.0]},
    ],
    evaluation_function=train_and_evaluate,
    objective_name='accuracy',
)

[INFO 08-01 14:35:49] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter lr. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 08-01 14:35:49] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter momentum. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 08-01 14:35:49] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='lr', parameter_type=FLOAT, range=[1e-06, 0.4], log_scale=True), RangeParameter(name='momentum', parameter_type=FLOAT, range=[0.0, 1.0])], parameter_constraints=[]).
[INFO 08-01 14:35:49] ax.modelbridge.dispatch_utils: Using Models.GPEI since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 08-01 14:35:49] ax.modelbridg